In [1]:
# set up matplotlib
%matplotlib widget
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams.update({"font.size": 14})

import numpy as np
import xarray as xr
from scipy.special import j0
from scipy.interpolate import RectBivariateSpline
from xbout import open_boutdataset

In [2]:
cases = ["mms1","mms2","mms3"]
grids = ["lowest", "low", "mid"]
data = []
ncs = []
for c, case in enumerate(cases):
    filepath = str(case) + "/BOUT.dmp.*.nc"

    ds = open_boutdataset(datapath=filepath, chunks={"t": 4})
    dsn = xr.open_dataset("circle_nocut_" + str(grids[c]) + "res.fci.nc")

    # Use squeeze() to get rid of the y-dimension, which has length 1 usually unless turbulent.
    ds = ds.squeeze(drop=True)
    dsn = dsn.squeeze(drop=True)

    dx = ds["dx"].isel(z=0).values
    # Get rid of existing "x" coordinate, which is just the index values.
    #ds = ds.drop("x")
    #Create a new coordinate, which is length in units of rho_s
    ds = ds.assign_coords(x=np.arange(ds.sizes["x"])*dx)

    data.append(ds)
    ncs.append(dsn)

/home/tirkas1/miniconda3/lib/python3.13/site-packages/xbout/load.py:341: UserWarning: No geometry type found, no physical coordinates will be added
  warn("No geometry type found, no physical coordinates will be added")


Read in:
<xbout.BoutDataset>
Contains:
<xarray.Dataset> Size: 46MB
Dimensions:             (x: 68, y: 1, z: 68, t: 201)
Coordinates:
    dx                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    dy                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    dz                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
  * t                   (t) float64 2kB 0.0 5e+05 1e+06 ... 9.95e+07 1e+08
  * x                   (x) int64 544B 0 1 2 3 4 5 6 7 ... 61 62 63 64 65 66 67
  * y                   (y) float64 8B 0.1963
  * z                   (z) float64 544B 0.007353 0.02206 ... 0.9779 0.9926
Data variables: (12/52)
    Bxy                 (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    G1                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 68), meta=np.ndarray>
    G2                  (x, y, z) float64 37kB dask.array<chunksize=(68, 1, 

/home/tirkas1/miniconda3/lib/python3.13/site-packages/xbout/load.py:341: UserWarning: No geometry type found, no physical coordinates will be added
  warn("No geometry type found, no physical coordinates will be added")


Read in:
<xbout.BoutDataset>
Contains:
<xarray.Dataset> Size: 172MB
Dimensions:             (x: 132, y: 1, z: 132, t: 201)
Coordinates:
    dx                  (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
    dy                  (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
    dz                  (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
  * t                   (t) float64 2kB 0.0 5e+05 1e+06 ... 9.95e+07 1e+08
  * x                   (x) int64 1kB 0 1 2 3 4 5 6 ... 126 127 128 129 130 131
  * y                   (y) float64 8B 0.1963
  * z                   (z) float64 1kB 0.003788 0.01136 ... 0.9886 0.9962
Data variables: (12/52)
    Bxy                 (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
    G1                  (x, y, z) float64 139kB dask.array<chunksize=(132, 1, 132), meta=np.ndarray>
    G2                  (x, y, z) float64 139kB dask.array<

/home/tirkas1/miniconda3/lib/python3.13/site-packages/xbout/load.py:341: UserWarning: No geometry type found, no physical coordinates will be added
  warn("No geometry type found, no physical coordinates will be added")


In [3]:

# Inputs assumed:
# bdy_pts: (Nb, 2) array, columns [R_b, Z_b] for boundary intercepts (BI)
# img_pts: (Nb, 2) array, columns [R_ip, Z_ip] for image points (inside fluid)
# dist_n:  (Nb,) array, distance along normal from BI to IP (physical coordinates)
# x_spl, z_spl: splines such that x_spl.ev(R,Z), z_spl.ev(R,Z) give computational coords
# x, z: 1D arrays of computational grid coords (same indexing as dens)
# dens: array with shape (nt, nx, nz) in (t,x,z) with indexing='ij'
# t_array: 1D array of physical times
# analytic_density(R, Z, t): function returning analytic density at given physical coords

def neumann_boundary_error_at_time(bdy_pts, img_pts, normals, spl, bc_val):
    # --- 1. Physical boundary & image points ---
    Rb = bdy_pts[:, 0]
    Zb = bdy_pts[:, 1]
    Rip = img_pts[:, 0]
    Zip = img_pts[:, 1]

    # --- 2. Map to computational coordinates (x,z) ---
    spl_e = spl #RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[k,:,:], kx=1, ky=1)
    n_num_b  = spl_e.ev(Rb,  Zb)
    n_num_ip  = spl_e.ev(Rip, Zip)

    # --- 4. Numerical normal derivative (one-sided) ---
    # du/dn ≈ (u(IP) - u(BI)) / dist_n
    du_dn_num = (n_num_ip - n_num_b) / np.hypot(normals[:,0], normals[:,1])

    # --- 5. Analytic normal derivative (same stencil) ---
    #t = t_array[t_index]

    #n_ana_b  = analytic_density(Rb,  Zb,  t)
    #n_ana_ip = analytic_density(Rip, Zip, t)

    du_dn_ana = bc_val #(n_ana_ip - n_ana_b) / normals #TODO: Get BC condition?

    # If Neumann BC is zero-flux, du_dn_ana should be ~0; you can
    # just look at |du_dn_num|. Otherwise compare num vs ana:

    err = du_dn_num - du_dn_ana

    # L2 and Linf norms along the boundary
    L2_bnd   = np.sqrt(np.nanmean(err**2))
    Linf_bnd = np.nanmax(np.abs(err))

    # Relative norms (guard against zero analytic derivative)
    denom_L2   = max(np.nanmean(np.abs(du_dn_ana)), 1e-14)
    denom_Linf = max(np.nanmax(np.abs(du_dn_ana)), 1e-14)

    L2_rel   = L2_bnd   / denom_L2
    Linf_rel = Linf_bnd / denom_Linf

    return {
        "du_dn_num": du_dn_num,
        "du_dn_ana": du_dn_ana,
        "err": err,
        "L2_bnd": L2_bnd,
        "Linf_bnd": Linf_bnd,
        "L2_rel": L2_rel,
        "Linf_rel": Linf_rel,
    }

def dirichlet_boundary_error_at_time(bdy_pts, spl, bc_val):
    # --- 1. Physical boundary & image points ---
    Rb = bdy_pts[:, 0]
    Zb = bdy_pts[:, 1]

    # --- 2. Map to computational coordinates (x,z) ---
    #spl = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[k,:,:], kx=1, ky=1)
    n_num_b  = spl.ev(Rb,  Zb)

    # --- 5. Analytic normal derivative (same stencil) ---
    #t = t_array[t_index]

    #n_ana_b  = analytic_density(Rb,  Zb,  t)
    #n_ana_ip = analytic_density(Rip, Zip, t)

    n_ana = bc_val #(n_ana_ip - n_ana_b) / normals #TODO: Get BC condition?

    # If Neumann BC is zero-flux, du_dn_ana should be ~0; you can
    # just look at |du_dn_num|. Otherwise compare num vs ana:

    err = n_num_b - n_ana

    # L2 and Linf norms along the boundary
    L2_bnd   = np.sqrt(np.nanmean(err**2))
    Linf_bnd = np.nanmax(np.abs(err))

    # Relative norms (guard against zero analytic derivative)
    denom_L2   = max(np.nanmean(np.abs(n_ana)), 1e-14)
    denom_Linf = max(np.nanmax(np.abs(n_ana)), 1e-14)

    L2_rel   = L2_bnd   / denom_L2
    Linf_rel = Linf_bnd / denom_Linf

    return {
        "n_num_b": n_num_b,
        "n_ana": n_ana,
        "err": err,
        "L2_bnd": L2_bnd,
        "Linf_bnd": Linf_bnd,
        "L2_rel": L2_rel,
        "Linf_rel": Linf_rel,
    }

In [ ]:
import numpy as np
from scipy.interpolate import RegularGridInterpolator
def make_fluid_near_ghost_mask(inside, ghost):
    """
    Return a mask for fluid cells that have at least one ghost neighbor
    in +/-x or +/-z.
    """

    # Start with no neighbors marked
    nx, nz = inside.shape

    ghost_xp = np.zeros_like(ghost, dtype=bool)
    ghost_xm = np.zeros_like(ghost, dtype=bool)
    ghost_zp = np.zeros_like(ghost, dtype=bool)
    ghost_zm = np.zeros_like(ghost, dtype=bool)

    # Shift ghost mask to find neighbors
    ghost_xp[:-1, :] = ghost[1:, :]   # neighbor at +x
    ghost_xm[1:,  :] = ghost[:-1, :]  # neighbor at -x
    ghost_zp[:, :-1] = ghost[:, 1:]   # neighbor at +z
    ghost_zm[:, 1:]  = ghost[:, :-1]  # neighbor at -z

    has_ghost_neighbor = ghost_xp | ghost_xm | ghost_zp | ghost_zm

    # We only care about FLUID cells that see a ghost
    mask_fluid_near_ghost = inside.astype(bool) & has_ghost_neighbor.astype(bool)
    return mask_fluid_near_ghost

def l2_error_near_ghost_cells(dens, dens_true, inside, ghost, V=None, t_index=-1):
    """
    L2 error of density at time t_index, restricted to fluid cells
    that have at least one ghost neighbor in x or z.
    """
    if V is None:
        V = np.ones_like(inside, dtype=float)

    mask_edge = make_fluid_near_ghost_mask(inside, ghost)
    #Switch to all non ghost cells.
    xsz = np.shape(dens[0,:,:].values)[0]
    zsz = np.shape(dens[0,:,:].values)[1]
    #Make slices for very inner cells and use t_ind = 1 always to check internal early convergence.
    xcut = xsz//2-(xsz//64)*4
    zcut = zsz//2-((zsz-4)//64)*4
    
    dens_diff = np.abs(dens[t_index].where(mask_edge)-dens_true[t_index].where(mask_edge))
    l2_err = np.sqrt(np.nanmean(dens_diff**2))
    dens_diff_inner = np.abs(dens[1,xcut:-xcut,zcut:-zcut]-dens_true[1,xcut:-xcut,zcut:-zcut])
    l2_err_in = np.sqrt(np.nanmean(dens_diff_inner**2))

    return l2_err, mask_edge, l2_err_in

def interpolate_on_boundary(x, z, density, dens_true, xb, zb, method="linear", t_index=-1):
    """
    Interpolate numerical and true density onto boundary points.

    Parameters
    ----------
    x, z : 1D arrays
        Grid coordinates in x and z directions.
    density, dens_true : 2D arrays, shape (len(x), len(z))
        Numerical and reference ("true") fields at a fixed time.
    xb, zb : arrays of same shape
        Boundary point coordinates (x_b, z_b).
    method : {"linear", "nearest"}, optional
        Interpolation method for RegularGridInterpolator.
        ("linear" ~ bilinear in 2D)

    Returns
    -------
    dens_b : array, same shape as xb
        Interpolated numerical density at boundary.
    dens_true_b : array, same shape as xb
        Interpolated true density at boundary.
    err_b : array, same shape as xb
        Pointwise error dens_b - dens_true_b.
    """
    # Flatten boundary points to (N, 2)
    pts = np.column_stack((xb.ravel(), zb.ravel()))

    interp_num = RegularGridInterpolator(
        (x, z), density[t_index,:,:].values,
        method=method,
        bounds_error=False,
        fill_value=None,  # extrapolate if slightly outside
    )
    interp_true = RegularGridInterpolator(
        (x, z), dens_true[t_index,:,:].values,
        method=method,
        bounds_error=False,
        fill_value=None,
    )

    dens_b_flat = interp_num(pts)
    dens_true_b_flat = interp_true(pts)

    dens_b = dens_b_flat.reshape(xb.shape)
    dens_true_b = dens_true_b_flat.reshape(xb.shape)
    err_b = dens_b - dens_true_b
    
    err_flat = err_b.ravel()
    l2 = np.sqrt(np.mean(err_flat**2))
    linf = np.max(np.abs(err_flat))
    return l2,linf


In [ ]:
R0s = [0.5, 0.5, 0.5]
Z0s = [0.4926470588235294, 0.4962121212121212, 0.4980769230769231]
alpha0 = 2.404825557693792
a = 1/3
a_phys = a #1.0
D = 1.0
m = 1.0
n = 1.0
bc_val = 1.0
timestep_for_errs_early = 10
timestep_for_errs_late = 100

w_unit = 95788333.030660808 #Omega_ci
t_unit = 1/w_unit

denses = []
dens_trues = []
masks = []
ghost_masks = []
dens_diffs = []
num_errs = []
l2_errs = []
linf_errs = []
l2_edges = []

lbda_errs = np.zeros_like(R0s)
l2_bd_err = []
linf_bd_err = []
l2_inner_errs = []
mass_changes = []
mass_errs = []

for d, dat in enumerate(data):
    print("Reading data for run: ", d)
    dens = dat["Nh+"][:,2:-2,:]
    src = dat["SNh+"][:,2:-2,:]
    dens_true = dens.copy()

    mask = ncs[d]["in_mask"][2:-2,:]
    #vols = ncs[d]["vol_frac"]
    ghost_id = ncs[d]["ghost_id"][2:-2,:].values
    ghost_mask = (ghost_id >= 0).astype(bool)
    full_mask = ghost_mask | mask.astype(bool)
    #vols = np.where(mask, vols, 0)
    gst_pts = ncs[d]["ghost_pts"].values
    bdy_pts = ncs[d]["bndry_pts"].values
    img_pts = ncs[d]["image_pts"].values
    normals = ncs[d]["normals"].values
    is_plasma = ncs[d]["is_plasma"].values
    Rarr = ncs[d]["R"][2:-2,:].values
    Zarr = ncs[d]["Z"].values

    spl_e = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[timestep_for_errs_early,:,:], kx=1, ky=1)
    spl_l = RectBivariateSpline(Rarr[:,0], Zarr[0,:], dens.values[timestep_for_errs_late,:,:],  kx=1, ky=1)

    nx = 64*(2**(d))
    nz = nx + 4
    dx = np.float64(1.0) / np.float64(nx)
    dz = np.float64(1.0) / np.float64(nz)

    # Centers in x; faces/nodes in z if that's your convention
    x = (np.arange(nx, dtype=np.float64) + 0.5) * dx     # centered x
    z = (np.arange(nz, dtype=np.float64)) * dz           # unshifted z

    #ghosts_lo_x = x[0]  - dx*np.arange(2, 0, -1)
    #ghosts_hi_x = x[-1] + dx*np.arange(1, 2+1)
    #x = np.concatenate((ghosts_lo_x, x, ghosts_hi_x))
    xx,zz = np.meshgrid(x,z,indexing='ij')
    
    xhalf  = np.float64(x[len(x)//2])
    zhalf  = np.float64(z[len(z)//2])
    
    R_min, R_max = ncs[d]["R"].values[0,0], ncs[d]["R"].values[-1,0]
    Z_min, Z_max = ncs[d]["Z"].values[0,0], ncs[d]["Z"].values[0,-1]
    dR = ncs[d]["R"].values[1,0] - ncs[d]["R"].values[0,0]
    dZ = ncs[d]["Z"].values[0,1] - ncs[d]["Z"].values[0,0]

    x0 = R0s[d]
    z0 = Z0s[d]

    t64 = np.asarray(ds["t_array"].values, dtype=np.float64) # 1D time
    t_norm = t64*t_unit

    rho = np.hypot((xx - xhalf), (zz - zhalf)) #(R_max-R_min)*, (Z_max-Z_min)*
    spatial = j0(alpha0 * rho / a)
    temp    = np.exp(-D * (alpha0 / a_phys)**2 * t_norm)

    #Compare simulation lambda to analytic value:
    #Use density near the center, which is always 1...
    n0 = dens.values[0,(nx+4)//2-1,nz//2-1]
    n1 = dens.values[1,(nx+4)//2-1,nz//2-1]
    dt = t_norm[1] - t_norm[0]
    #ratio = (n1-1) / (n0-1)
    ratio = (n1-bc_val)/(n0-bc_val)
    lambda_eff = -np.log(ratio) / (D * dt)
    lbda = (alpha0/a_phys)**2
    #print("Effective lambda from simulation after one timestep:", lambda_eff)
    #print("Analytic lambda:", lbda)
    lbda_errs[d] = 100*np.abs((lambda_eff - lbda))/lbda
    print(lbda)
    print(lambda_eff)
    #print("Percent error:", lbda_errs[d])

    dens_true = dens.copy()
    dens_true[:] = spatial[np.newaxis,:,:] * temp[:,np.newaxis,np.newaxis] + 1

    denses.append(dens)
    dens_trues.append(dens_true)
    masks.append(mask)
    ghost_masks.append(ghost_mask)

    # Example usage:
    res = dirichlet_boundary_error_at_time(bdy_pts, spl_e, bc_val)
    l2_bd_err.append(res["L2_bnd"])
    linf_bd_err.append(res["Linf_bnd"])

    #M0 = (dens.where(mask)*dR*dZ)[0,:,:].sum().values
    #M1 = (dens.where(mask)*dR*dZ)[10,:,:].sum().values
    #M2 = (dens.where(mask)*dR*dZ)[100,:,:].sum().values
    #M3 = (dens.where(mask)*dR*dZ)[-1,:,:].sum().values
    ##print(M0, M1, M2, M3)
    #mass_change = M3-M0
    #mass_changes.append(mass_change)
    #mass_errs.append(np.abs(mass_change)/M0)

    #Get L2 error
    N_valid = np.sum(mask)
    dens_diff = np.abs(dens.where(mask)-dens_true.where(mask))
    linf_err_t = np.max(dens_diff)
    l2_err_t = np.sqrt(np.nanmean((dens_diff**2), axis=(1, 2)))
    dens_diffs.append(dens_diff)
    l2_errs.append(l2_err_t)
    linf_errs.append(linf_err_t.values)

    init_dens_diff = dens_diff[0,:,:]
    mask2 = init_dens_diff.notnull() & (init_dens_diff >= 1e-10)
    nerrs = int(mask2.sum().compute())
    num_errs.append(nerrs)
    
    # Example usage:
    #V = vols*dR*dZ?
    L2_edge, edge_mask, l2_inner = l2_error_near_ghost_cells(dens, dens_true, mask, ghost_mask, V=dR*dZ, t_index=1)
    l2_edges.append(L2_edge)
    l2_inner_errs.append(l2_inner)

print('\n')
print("Eigenmode rel. err.: ", lbda_errs[0], lbda_errs[1], lbda_errs[2])
print("Eigenmode rel. err. convergence: ", lbda_errs[0]/lbda_errs[1], lbda_errs[1]/lbda_errs[2])

#print('\n')
#print("Mass change: ", mass_changes[0], mass_changes[1], mass_changes[2])
#print("Mass errs: ", mass_errs[0], mass_errs[1], mass_errs[2])

print('\n')
print("L2 Bdy Err: ", l2_bd_err[0], l2_bd_err[1], l2_bd_err[2])
print("L2 Bdy Err convergence: ", l2_bd_err[0]/l2_bd_err[1], l2_bd_err[1]/l2_bd_err[2])

print("L2 early innermost error: ", l2_inner_errs[0], l2_inner_errs[1], l2_inner_errs[2])
print("L2 convergence in early innermost cells: ", l2_inner_errs[0]/l2_inner_errs[1], l2_inner_errs[1]/l2_inner_errs[2])

print("L2 relative error in fluid cells touching ghosts: ", l2_edges[0], l2_edges[1], l2_edges[2])
print("L2 convergence in fluid cells touching ghosts: ", l2_edges[0]/l2_edges[1], l2_edges[1]/l2_edges[2])

print('\n')
print("Linf Bdy Err: ", linf_bd_err[0], linf_bd_err[1], linf_bd_err[2])
print("Linf Bdy Err. convergence: ", linf_bd_err[0]/linf_bd_err[1], linf_bd_err[1]/linf_bd_err[2])

print('\n')
print("Cells different than analytic initial value: ", num_errs[0], num_errs[1], num_errs[2])

print('\n')
print("Linf errs: ", linf_errs[0], linf_errs[1], linf_errs[2])
print("Linf convergence: ", linf_errs[0]/linf_errs[1], linf_errs[1]/linf_errs[2])

print('\n')
print("L2 max errs: ", np.max(l2_errs[0]), np.max(l2_errs[1]), np.max(l2_errs[2]))
print("L2 early errs: ", l2_errs[0][timestep_for_errs_early], l2_errs[1][timestep_for_errs_early], l2_errs[2][timestep_for_errs_early])
print("L2 late errs: ", l2_errs[0][timestep_for_errs_late], l2_errs[1][timestep_for_errs_late], l2_errs[2][timestep_for_errs_late])
print("L2 final errs: ", l2_errs[0][-1], l2_errs[1][-1], l2_errs[2][-1])
print("L2 max convergence: ", np.max(l2_errs[0])/np.max(l2_errs[1]), np.max(l2_errs[1])/np.max(l2_errs[2]))


Reading data for run:  0
52.04867366643532
52.01064218740098
cuts:  28 30
Reading data for run:  1
52.04867366643532
52.038857948065306
cuts:  56 58
Reading data for run:  2
52.04867366643532
52.046258937642484
cuts:  112 114


Eigenmode rel. err.:  0.07306906469523175 0.01885872910599007 0.004639366621157462
Eigenmode rel. err. convergence:  3.8745487187693333 4.064936153134856


L2 Bdy Err:  2.6301734909707425e-05 6.137857917123343e-06 1.7280809843480266e-06
L2 Bdy Err convergence:  4.285165161013433 3.551834649369193
L2 relative error in fluid cells touching ghosts:  0.00014441431567118994 3.719548074784917e-05 9.062951195535574e-06
L2 convergence in fluid cells touching ghosts:  3.8825769358967275 4.10412457767308


Linf Bdy Err:  5.296320869840887e-05 1.5264654063540384e-05 4.15688550658988e-06
Linf Bdy Err. convergence:  3.4696632152910336 3.672137238165795


Cells different than analytic initial value:  0 0 0


Linf errs:  0.000566935782898792 0.000148430212755013 6.356007565422

In [ ]:
denses[2].where(ghost_masks[2]).bout.animate2D(aspect='equal')

In [ ]:
plt.plot(t_norm, l2_errs[0])
plt.xlabel("Time [s]")
plt.ylabel("L2 error (RMS)")
plt.title("Time evolution of L2 error")
plt.show()